# Классификация клиентов банка - прогнозирование кредитной истории

## Подключение модулей

In [452]:
# импортируем необходимые модули
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [454]:
%matplotlib inline

In [456]:
warnings.filterwarnings('ignore')

## Загрузка данных

In [639]:
# загружаем данные из файла в формате CSV, указываем символ-разделитель и сохраняем данные в дата-фрейм (переменную df)
df_train = pd.read_csv('train.csv', sep = ',', dtype='unicode')

In [175]:
# загружаем данные из файла в формате CSV, указываем символ-разделитель и сохраняем данные в дата-фрейм (переменную df)
df_test = pd.read_csv('test.csv', sep = ',', dtype='unicode')

In [ ]:
### В начале рассмотрим тренировачниый 

In [376]:
df_train.head(10)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333328,3,...,_,809.98,26.822619623699016,22 Years and 1 Months,No,49.57494921489417,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.94496005538421,NaN,No,49.57494921489417,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.60935202206993,22 Years and 3 Months,No,49.57494921489417,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377861869582354,22 Years and 4 Months,No,49.57494921489417,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333328,3,...,Good,809.98,24.797346908844986,22 Years and 5 Months,No,49.57494921489417,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,27.26225871052017,22 Years and 6 Months,No,49.57494921489417,62.430172331195294,!@9#%8,340.4792117872438,Good
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.8433333333328,3,...,Good,809.98,22.53759303178384,22 Years and 7 Months,No,49.57494921489417,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.8433333333328,3,...,Good,809.98,23.93379480196552,NaN,No,49.57494921489417,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986666666666,2,...,Good,605.03,24.46403063758457,26 Years and 7 Months,No,18.816214573128885,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,0x160f,CUS_0x21b1,February,Rick Rothackerj,28,004-07-5839,Teacher,34847.84,3037.986666666666,2,...,Good,605.03,38.550848433956325,26 Years and 8 Months,No,18.816214573128885,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good


### Описание данных

Признаки:

- ID – Уникальный идентификатор для каждой записи.
- Customer_ID – Уникальный идентификатор для каждого клиента.
- Month – Месяц транзакции или записи.
- Name – Имя клиента.
- Age – Возраст клиента.
- SSN – Номер социального страхования клиента.
- Occupation – Профессия клиента.
- Annual_Income – Годовой доход клиента.
- Monthly_Inhand_Salary – Месячная заработная плата клиента.
- Num_Bank_Accounts – Общее количество банковских счетов, принадлежащих клиенту.
- Num_Credit_Card – Общее количество кредитных карт, которыми владеет клиент.
- Interest_Rate – Процентная ставка, применяемая к займам или кредитам.
- Num_of_Loan – Количество кредитов, взятых клиентом.
- Type_of_Loan – Категории кредитов, полученных клиентом.
- Delay_from_due_date – Задержка платежей по сравнению с установленной датой.
- Num_of_Delayed_Payment – Общее количество случаев просрочки платежей, совершенных клиентом.
- Changed_Credit_Limit – Корректировки, внесенные в кредитный лимит клиента.
- Num_Credit_Inquiries – Количество запросов, сделанных в отношении кредита клиента.
- Credit_Mix – Разновидности кредитов, которыми пользуется клиент (например, кредиты, кредитные карты).
- Outstanding_Debt – Общая сумма задолженности клиента на данный момент.
- Credit_Utilization_Ratio – Доля использованного кредита по сравнению с общим кредитным лимитом.
- Credit_History_Age – Продолжительность кредитной истории клиента.
- Payment_of_Min_Amount – Указывает, платит ли клиент ежемесячно минимально необходимую сумму.
- Total_EMI_per_month – Общий эквивалентный ежемесячный платеж (EMI), выплаченный клиентом.
- Amount_invested_monthly – Ежемесячная сумма инвестиций клиента.
- Payment_Behaviour – Платежные привычки и склонности клиента.
- Monthly_Balance –  Остаток на счете клиента в конце каждого месяца.

Целевой (искомый) признак:
- Credit_Score –  Кредитная оценка клиента (целевая переменная: «Хорошая», «Плохая», «Стандартная»).

## Стадии решения задачи
- подключение модулей (import modules)
- загрузка данных (data lodaing)
- разведочный анализ данных (exploratory data analysis, EDA)
- предварительная обработка данных (preprocessing)
- обучение моделей (models fitting)
- прогнозирование (prediction)
- вывод (conclusion)

## Разведочный анализ данных (РАД, EDA)

In [378]:
# объём данных (строк и столбцов - объектов и признаков)
df_train.shape

(100000, 28)

In [380]:
# выводим типы данных
df_train.dtypes

ID                          object
Customer_ID                 object
Month                       object
Name                        object
Age                         object
SSN                         object
Occupation                  object
Annual_Income               object
Monthly_Inhand_Salary       object
Num_Bank_Accounts           object
Num_Credit_Card             object
Interest_Rate               object
Num_of_Loan                 object
Type_of_Loan                object
Delay_from_due_date         object
Num_of_Delayed_Payment      object
Changed_Credit_Limit        object
Num_Credit_Inquiries        object
Credit_Mix                  object
Outstanding_Debt            object
Credit_Utilization_Ratio    object
Credit_History_Age          object
Payment_of_Min_Amount       object
Total_EMI_per_month         object
Amount_invested_monthly     object
Payment_Behaviour           object
Monthly_Balance             object
Credit_Score                object
dtype: object

In [131]:
# целевой признак - Credit_Score

In [382]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ID                        100000 non-null  object
 1   Customer_ID               100000 non-null  object
 2   Month                     100000 non-null  object
 3   Name                      90015 non-null   object
 4   Age                       100000 non-null  object
 5   SSN                       100000 non-null  object
 6   Occupation                100000 non-null  object
 7   Annual_Income             100000 non-null  object
 8   Monthly_Inhand_Salary     84998 non-null   object
 9   Num_Bank_Accounts         100000 non-null  object
 10  Num_Credit_Card           100000 non-null  object
 11  Interest_Rate             100000 non-null  object
 12  Num_of_Loan               100000 non-null  object
 13  Type_of_Loan              88592 non-null   object
 14  Delay

Признаки Name, Monthly, Inhand_Salary, Type_of_Loan, Num_of_Delayed_Payment, Num_Credit_Inquiries, Credit_History_Age, Amount_invested_monthly, Monthly_Balance имеют пустые значения

Все признаки представлены в типе данных object. Необходимо привести соответсвующие парматеры к соотвутсвующему типу данных

In [384]:
# Посмотрим в процентном соотношении, сколько пропусков
for col in df_train.columns:
    print(f'Кол-во пропусков в колонке {col} = {df[col].isna().sum()}')

Кол-во пропусков в колонке ID = 0
Кол-во пропусков в колонке Customer_ID = 0
Кол-во пропусков в колонке Month = 0
Кол-во пропусков в колонке Name = 9985
Кол-во пропусков в колонке Age = 0
Кол-во пропусков в колонке SSN = 0
Кол-во пропусков в колонке Occupation = 0
Кол-во пропусков в колонке Annual_Income = 0
Кол-во пропусков в колонке Monthly_Inhand_Salary = 15002
Кол-во пропусков в колонке Num_Bank_Accounts = 0
Кол-во пропусков в колонке Num_Credit_Card = 0
Кол-во пропусков в колонке Interest_Rate = 0
Кол-во пропусков в колонке Num_of_Loan = 0
Кол-во пропусков в колонке Type_of_Loan = 11408
Кол-во пропусков в колонке Delay_from_due_date = 0
Кол-во пропусков в колонке Num_of_Delayed_Payment = 7002
Кол-во пропусков в колонке Changed_Credit_Limit = 0
Кол-во пропусков в колонке Num_Credit_Inquiries = 1965
Кол-во пропусков в колонке Credit_Mix = 0
Кол-во пропусков в колонке Outstanding_Debt = 0
Кол-во пропусков в колонке Credit_Utilization_Ratio = 0
Кол-во пропусков в колонке Credit_Histor

In [140]:
# Посмотрим на дублирующие значения
df_train.duplicated().sum()

0

Дублирующие значения отсутствуют

In [386]:
# описательная статистика для всех признаков
df_train.describe().T

,count,unique,top,freq
ID,100000,100000,0x1602,1
Customer_ID,100000,12500,CUS_0xd40,8
Month,100000,8,January,12500
Name,90015,10139,Langep,44
Age,100000,1788,38,2833
SSN,100000,12501,#F%$D@*&8,5572
Occupation,100000,16,_______,7062
Annual_Income,100000,18940,36585.12,16
Monthly_Inhand_Salary,84998,13235,6769.13,15
Num_Bank_Accounts,100000,943,6,13001


In [197]:
# Посмотрим уникальные значения у параметра Payment_of_Min_Amount
df_train.Payment_of_Min_Amount.unique()

array(['No', 'NM', 'Yes'], dtype=object)

In [238]:
# Посмотрим уникальные значения у параметра Age
df_train.Age.unique()

array(['23', '-500', '28_', ..., '4808_', '2263', '1342'], dtype=object)

In [ ]:
# Посмотрим уникальные значения у параметра Credit_Mix
df_train.Credit_Mix.unique()

In [213]:
# Посмотрим уникальные значения у параметра Occupation
df_train.Occupation.unique()

array(['Scientist', '_______', 'Teacher', 'Engineer', 'Entrepreneur',
       'Developer', 'Lawyer', 'Media_Manager', 'Doctor', 'Journalist',
       'Manager', 'Accountant', 'Musician', 'Mechanic', 'Writer',
       'Architect'], dtype=object)

In [217]:
# Посмотрим уникальные значения у параметра Month
df_train.Month.unique()

array(['January', 'February', 'March', 'April', 'May', 'June', 'July',
       'August'], dtype=object)

In [226]:
# Посмотрим уникальные значения у параметра Monthly_Balance
df_train.Monthly_Balance.unique()

array(['312.49408867943663', '284.62916249607184', '331.2098628537912',
       ..., '516.8090832742814', '319.1649785257098', '393.6736955618808'],
      dtype=object)

In [201]:
# Посмотрим уникальные значения у целевого параметра Credit_Score
df_train.Credit_Score.unique()

array(['Good', 'Standard', 'Poor'], dtype=object)

In [635]:
df_train

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,Short_Month
0,January,23.000000,Scientist,19114.12,1824.8433333333328,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,26.822619623699016,22 Years and 1 Months,No,49.57494921489417,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good,1
1,February,23.000000,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,31.94496005538421,NaN,No,49.57494921489417,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good,2
2,March,34.735535,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,28.60935202206993,22 Years and 3 Months,No,49.57494921489417,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good,3
3,April,23.000000,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,31.377861869582354,22 Years and 4 Months,No,49.57494921489417,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good,4
4,May,23.000000,Scientist,19114.12,1824.8433333333328,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",...,809.98,24.797346908844986,22 Years and 5 Months,No,49.57494921489417,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,April,25.000000,Mechanic,39628.99,3359.415833333333,4,6,7,2,"Auto Loan, and Student Loan",...,502.38,34.66357180929323,31 Years and 6 Months,No,35.10402261040746,60.97133255718485,High_spent_Large_value_payments,479.86622816574095,Poor,4
99996,May,25.000000,Mechanic,39628.99,3359.415833333333,4,6,7,2,"Auto Loan, and Student Loan",...,502.38,40.565630963889205,31 Years and 7 Months,No,35.10402261040746,54.18595028760385,High_spent_Medium_value_payments,496.651610435322,Poor,5
99997,June,25.000000,Mechanic,39628.99,3359.415833333333,4,6,5729,2,"Auto Loan, and Student Loan",...,502.38,41.255522259019116,31 Years and 8 Months,No,35.10402261040746,24.02847744864441,High_spent_Large_value_payments,516.8090832742814,Poor,6
99998,July,25.000000,Mechanic,39628.99,3359.415833333333,4,6,7,2,"Auto Loan, and Student Loan",...,502.38,33.63820798055965,31 Years and 9 Months,No,35.10402261040746,251.67258219721603,Low_spent_Large_value_payments,319.1649785257098,Standard,7


### Предварительная обработка данных

In [641]:
# Уберём столбы ID, Customer_ID, Name, SSN
df_train.drop(['ID', 'Customer_ID', 'Name', 'SSN'], axis = 1, inplace = True)

In [643]:
# Приведём параметр Month, к типу данных int
dict_month = {'January' : 1, 'February' : 2, 'March' : 3, 'April' : 4, 'May' : 5, 'June' : 6, 'July' : 7, 'August' : 8}
df_train['Short_Month'] = df_train['Month'].map(dict_month)

In [645]:
def check_to_number(x):
    if x.isdigit():
        return int(x)
    return np.nan

In [647]:
# Приведём параметр Age, к типу данных int
df_train['Age'] = df_train['Age'].map(check_to_number)

In [649]:
# Просмотрим на аномальные значения параметр Age
df_train.Age.describe()

count    94175.000000
mean       116.682187
std        690.079845
min         14.000000
25%         25.000000
50%         33.000000
75%         42.000000
max       8698.000000
Name: Age, dtype: float64

Есть аномальные значения, людям с кредитами не может быть меньше 18 лет и не может быть больше 100 лет. По этому заменим на всез кому меньше 18 лет, на 18 лет и кому больше 100 лет заменим на 100.

In [652]:
def check_to_age(x):
    if x < 18:  
        return 18
    elif x > 100:
        return 100
    return x

In [654]:
df_train['Age'] = df_train['Age'].map(check_to_age)

In [656]:
# Просмотрим на аномальные значения параметр Age
df_train.Age.describe()

count    94175.000000
mean        34.735535
std         13.874516
min         18.000000
25%         25.000000
50%         33.000000
75%         42.000000
max        100.000000
Name: Age, dtype: float64

In [658]:
# Просмотрим на пустые значения параметр Age
df_train.Age.isna().sum()

5825

In [660]:
# Заменим пустые значения на медианые
df_train.Age.fillna(df_train.Age.mean(), inplace=True)

In [662]:
# Проверим на пустые значения параметр Age
df_train.Age.isna().sum()

0

In [664]:
df_train.Occupation.value_counts()

Occupation
_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64

In [666]:
df_train['Occupation'] = df_train['Occupation'].str.replace('_______', 'No occupation')

In [668]:
df_train.Occupation.unique()

array(['Scientist', 'No occupation', 'Teacher', 'Engineer',
       'Entrepreneur', 'Developer', 'Lawyer', 'Media_Manager', 'Doctor',
       'Journalist', 'Manager', 'Accountant', 'Musician', 'Mechanic',
       'Writer', 'Architect'], dtype=object)

In [ ]:
# изменим значения в столбце Annual_Income, в которых есть '_'


In [679]:
# Приведём столбец Annual_Income к численному типу
df_train.Annual_Income = df_train.Annual_Income.map(lambda x: float(x))

ValueError: could not convert string to float: '34847.84_'

In [675]:
df_train.Age.describe()

count    100000.000000
mean         34.735535
std          13.464354
min          18.000000
25%          25.000000
50%          34.000000
75%          42.000000
max         100.000000
Name: Age, dtype: float64